In [288]:
import pandas as pd
import json
from bs4 import BeautifulSoup,NavigableString, Tag

In [327]:
# import csv file to pandas dataframe
df = pd.read_csv('allData.csv')
df.drop(columns=df.columns[0], axis=1, inplace=True)

In [531]:
df.loc[df['name'] == "Bipolar disorder"]

,name,link,description,header
33,Bipolar disorder,https://www.nhsinform.scot/illnesses-and-condi...,"<div class=""js-guide cf guide"">\n<div class=""c...","[{'title': 'Introduction', 'data': ['<div clas..."


In [248]:
# Remove '[]' symbol in description and get appropriate data
# Do not run again
formatted_description = []
for index, row in df.iterrows():
   fData = row['description'][1:-1]
   soup = BeautifulSoup(fData, 'html.parser')
   mainData = soup.find('div', 'js-guide cf guide')
   if mainData==None:
      editors = soup.find_all('div','editor')
      mainData = ''
      for editor in editors:
         mainData += str(editor)
      mainData = '>div class="js-guide cf guide">' + mainData + '</div>'
   else:
      mainData = str(mainData)
   formatted_description.append(mainData)
df['description'] = pd.Series(formatted_description)

In [320]:
# All headers and their content have been added to the dataframe
# No need to run again
header_words = []
for i,d in df.iterrows():
   soup = BeautifulSoup(d['description'], 'html.parser')
   headers = soup.find_all('h2')
   header_and_data = []
   for header in headers:
      item = {'title':header.text, 'data':[]}
      for sib in header.find_next_siblings():
         if(sib.name=="h2"):
            break
         else:
            item['data'].append(str(sib))
      header_and_data.append(item)
   header_words.append(header_and_data)
header_words = pd.Series(header_words)
df['header'] = header_words

In [347]:
print(any(word in 'This is a test' for word in ['isa']))

False


In [526]:
def determine_typeOfTitle(title):
   if(any(word in title for word in ['symptom','happen','sign','indicator','feature','trait','clue','problem','stroke','pain','feel'])):
      return 'symptom'
   if(any(word in title for word in ['about','defini','descrip','explana','explain','information','introduct','how common','how long','how many','how much'])):
      return 'about'
   elif(any(word in title for word in ['cause','trigger','origin','source','spread','basis','reason','factors''infected'])):
      return 'cause'
   if(any(word in title for word in ['bite','cyst','how do i get'])):
      return 'cause'
   if(any(word in title for word in ['diagnos','Immunophenotyping','pcr','scan','x-ray','test','identify','identifi','detect','discover','screen','support'])):
      return 'diagnosis'
   if(any(word in title for word in ['prevent','rule','precaution','control','mitigation','diet','exercise','tips'])):
      return 'prevention'
   if(any(word in title for word in ['tablet','induction','imatinib','treat','therap','therapeutic','inject','medicat','medicine','consolidat','mainten','biopsy','painkill','surger','etymology','scopy','ology','biotics','puncture','transplant'])):
      return 'treatment'
   if(any(word in title for word in ['who','risk','blood pressure','smoking','history','family','weight','famili','men','women','old','young','depress','genetic','radiation','environment','benzene','alcohol','age'])):
      return 'risks'
   if(any(word in title for word in ['complicat','bleed','infertil','effect','peritonitis','abscess','infect'])):
      return 'complications'
   if(any(word in title for word in ['vaccin'])):
      return 'vaccination'
   if(any(word in title for word in ['medical advice','advice','hospital','getting help','living with'])):
      return 'medical advice'
   if(any(word in title for word in ['emergency'])):
      return 'emergency'
   if(any(word in title for word in ['type','class','category','stage'])):
      return 'types'
   if(any(word in title for word in ['myth'])):
      return 'myths'
   else:
      return 'other'

In [527]:
all_header_types = []
data = []
for i,d in df.iterrows():
   headers = eval(d['header'])
   item = []
   for header in headers:
      title=header['title']
      type = determine_typeOfTitle(title.lower())
      # print(type)
      item.append({'title':title, 'type':type, 'data':header['data'],'disease':d['name']})
      all_header_types.append(type)
   data.extend(item)
all_header_types = pd.Series(all_header_types)
all_header_types.value_counts()

other             1540
treatment         1009
symptom            600
cause              528
diagnosis          507
risks              446
about              339
complications      234
prevention         225
medical advice     214
types               53
vaccination         20
emergency            1
myths                1
dtype: int64

In [ ]:
data = sorted(list(filter(lambda x: len(x['title']) and x['type'],data)), key=lambda x: x['title'])
for d in data:
   if(d['disease']=='Bipolar disorder' and d['type']=='other'):
      print(d['title']+'\n'+d['type']+"-"+d['disease']+'\n'+str(d['data']))  
# result = list(map(lambda x: x['type']+" : "+x['title']+" : "+x['disease'],data))
# sorted(result)[1690:]

In [545]:
allData = pd.json_normalize(data)
allData = allData.sort_values(by=['disease','type']).reset_index(drop=True)
allData = allData.reindex(columns=['disease','title','type','data'])
allData.head()

,disease,title,type,data
0,Abdominal aortic aneurysm,About abdominal aortic aneurysms,about,"[<div class=""editor"">\n<p>An abdominal aortic ..."
1,Abdominal aortic aneurysm,Causes of an AAA,cause,[<p>It's not known exactly what causes the aor...
2,Abdominal aortic aneurysm,Causes of an abdominal aortic aneurysm,cause,"[<div class=""editor"">\n<p>The aorta is the lar..."
3,Abdominal aortic aneurysm,Diagnosing an AAA,diagnosis,"[<p>Because AAAs usually cause no symptoms, th..."
4,Abdominal aortic aneurysm,Diagnosing an abdominal aortic aneurysm,diagnosis,"[<div class=""editor"">\n<p>An abdominal aortic ..."


In [546]:
allData.loc[allData['type'] == 'other']

,disease,title,type,data
8,Abdominal aortic aneurysm,If you have a large AAA,other,[<p>If you are diagnosed with an AAA that is 5...
9,Abdominal aortic aneurysm,If you have a small or medium AAA,other,[<p>If you are diagnosed with a small (3.0-4.4...
10,Abdominal aortic aneurysm,Ruptured AAA,other,"[<p>If your aortic aneurysm ruptures, you will..."
32,Acne,Isotretinoin,other,[<p>Isotretinoin has a number of beneficial ef...
33,Acne,What can I do if I have acne?,other,[<p>The self-help techniques below may be usef...
...,...,...,...,...
5685,Womb (uterus) cancer,Dealing with dying,other,[<p>If you're told nothing more can be done to...
5686,Womb (uterus) cancer,Follow-up,other,[<p>After your course of treatment has finishe...
5687,Womb (uterus) cancer,Relationships and sex,other,[<h3>Relationships with friends and family</h3...
5688,Womb (uterus) cancer,Talk to others,other,"[<p>Being diagnosed with cancer can be hard, b..."


In [542]:
allData.head()

,title,type,data,disease
24,About abdominal aortic aneurysms,about,"[<div class=""editor"">\n<p>An abdominal aortic ...",Abdominal aortic aneurysm
735,Causes of an AAA,cause,[<p>It's not known exactly what causes the aor...,Abdominal aortic aneurysm
736,Causes of an abdominal aortic aneurysm,cause,"[<div class=""editor"">\n<p>The aorta is the lar...",Abdominal aortic aneurysm
1362,Diagnosing an AAA,diagnosis,"[<p>Because AAAs usually cause no symptoms, th...",Abdominal aortic aneurysm
1363,Diagnosing an abdominal aortic aneurysm,diagnosis,"[<div class=""editor"">\n<p>An abdominal aortic ...",Abdominal aortic aneurysm


In [326]:
df.to_csv('allData.csv')

In [537]:
json.dump(data, open('data.json', 'w'))